# **02443 - Computer Exercise 2: Sampling from Discrete Distributions**

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp
import plotly.io as pio
pio.renderers.default = "notebook+pdf"
pio.templates.default = "plotly_dark"

from utils import chi_sq_test, kolmogorov_smirnov_test

## **Part 1 - Simulation using $p$**

In [22]:
# Set p to a fixed value on the open interval between 0 and 1
p = 0.1
N = 10000

# Generate uniform random numbers
U = np.random.uniform(0, 1, N)

# Use formula from lecture 3 slide 9 to get geometric distributed discrete numbers
X = np.ceil(np.log(U)/np.log(1-p))

# Histogram
fig = go.Figure(go.Histogram(x=X, histnorm='probability density', marker_color='Green'))
fig.update_layout(
    title=f"Geometric Distribution with p={p}",
    xaxis_title="Number of Trials",
    yaxis_title="Density",
    width=600, height=400,
    bargap=0.1
)
fig.show()

## **Part 2 - Simulating 6-point distribution**

In [3]:
ps = np.array([7/48, 5/48, 1/8, 1/16, 1/4, 5/16])

### **(a) Direct method**

In [4]:
def direct(n, ps):
    # Generate uniform random numbers
    U = np.random.rand(N)

    # Convert to discrete random numbers using the given probabilities
    X = np.searchsorted(np.cumsum(ps), U)

    return X

### **(b) Rejection method**

In [5]:
def rejection(n, ps):
    c = max(ps)
    k = len(ps)

    X = np.zeros(n, dtype=int)
    for i in range(len(X)):
        while True: # Could theoretically run forever...
            U1, U2 = np.random.rand(2)
            I = np.floor(k * U1).astype(int)
            if U2 <= ps[I]/c:
                X[i] = I + 1
                break
    
    return X

### **(c) Alias method**

In [6]:
def alias(N, ps):
    k = len(ps)

    # Generating Alias tables
    L = np.arange(k)
    F = k*ps
    G = np.where(F >= 1)[0]
    S = np.where(F <= 1)[0]

    while len(S) != 0:
        i = G[0]
        j = S[0]
        L[j] = i
        F[i] -= (1 - F[j])
        if F[i] < 1 - np.finfo(float).eps:
            G = np.delete(G, 0)
            S = np.append(S, i)
        S = np.delete(S, 0)

    # Computing values
    X = np.zeros(N, dtype=int)

    # Generate random numbers
    U1 = np.random.rand(N)
    U2 = np.random.rand(N)

    # Perform Alias method
    I = np.array(np.floor(k * U1)).astype(int)
    mask = U2 <= F[I]
    X[mask] = I[mask] + 1
    X[~mask] = L[I[~mask]] + 1

    return X

## **Part 3 - Comparison**

### **Histograms**

In [7]:
X_d = direct(N, ps)
X_r = rejection(N, ps)
X_a = alias(N, ps)

# Plot histograms
fig = sp.make_subplots(rows=1, cols=4, subplot_titles=("True probabilities", "Direct Method", "Rejection Method", "Alias Method"))
fig.add_trace(go.Bar(x=np.arange(1, 7), y=ps, marker_color='green'), row=1, col=1) # True probabilities
fig.add_trace(go.Histogram(x=X_d, histnorm='probability density', marker_color='orange'), row=1, col=2) # Direct method
fig.add_trace(go.Histogram(x=X_r, histnorm='probability density', marker_color='blue'), row=1, col=3) # Rejection method
fig.add_trace(go.Histogram(x=X_a, histnorm='probability density', marker_color='red'), row=1, col=4) # Alias method
fig.update_layout(height=300, width=1000, showlegend=False, bargap=0.1, margin=dict(l=50, r=50, t=50, b=50))
fig.show()

All methods yield discrete random numbers that appear to be correctly distributed when inspecting the histogram. 

### **Tests**

In [8]:
methods = ["Direct", "Rejection", "Alias"]
print("-"*36)
for i, X in enumerate([X_d, X_r, X_a]):
    tab = np.array([chi_sq_test(X, ps), kolmogorov_smirnov_test(X, ps)])
    df = pd.DataFrame(np.round(tab, 2), index=["Chi squared", "Kol-Smi"], columns=["Test statistic", "p-value"])
    print(">>> " + methods[i] + " method <<<")
    print(df)
    print("-"*36)

------------------------------------
>>> Direct method <<<
             Test statistic  p-value
Chi squared            3.51     0.62
Kol-Smi                0.00     1.00
------------------------------------
>>> Rejection method <<<
             Test statistic  p-value
Chi squared            7.54     0.18
Kol-Smi                0.01     1.00
------------------------------------
>>> Alias method <<<
             Test statistic  p-value
Chi squared             1.6      0.9
Kol-Smi                 0.0      1.0
------------------------------------


For each method, we have calculated the p-value using both the chi squared test and the Kolmogorov Smirnov test. All p-values do not give us ground for rejecting the null hypothesis, i.e. that the data is drawn from the same distribution as the given probabilities.

## **Part 4 - Pros and Cons of the different methods**

The rejection method has the con that to achieve the desired number of samples then it potentially needs to run for many more iterations due to the rejections. Many rejections can happen when you have some probabilities that are very low and some probabilities that are quite high. With $c=\text{max}(p_i)$ then $p_i/c$ will also be quite low when $p_i$ is low. This means that the probability of accepting a sample is quite low.

In such a case then the alias method will be better as there is no rejection in the alias method. There is some overhead in the initialization of the alias method.